In [4]:
import pandas as pd
import numpy as np
import attancanceUtil as util


attendance_df = pd.read_csv('../data/matches.csv', sep=',', parse_dates=[1], date_parser=util.custom_date_parser)

# Technische NA (16 Spiele wurden nicht durchgeführt und haben keine gültigen Werte).
attendance_df.dropna(inplace=True)
attendance_df = attendance_df[(attendance_df['attendance'] != 'Unter Ausschluss der Öffentlichkeit')]

#Nummerische werte als solche definieren
attendance_df['attendance'] = pd.to_numeric(attendance_df['attendance'])
attendance_df['matchday'] = pd.to_numeric(attendance_df['matchday'], downcast='integer')


#saison auslesen:
season = attendance_df.apply(lambda x: util.determine_season(x['date']), axis=1)
attendance_df['season'] = season

#Hinzufügen der Geografischen Distanz:
team_distance = attendance_df.apply(lambda x: util.calc_dist(x['hometeam'], x['awayteam']), axis=1)
attendance_df['team_distance'] = team_distance


#Hinzufügen der Punkte vor der Runde:
points_home = attendance_df.apply(lambda x: util.calc_point_average_before_game(x['season'], x['matchday'],x['hometeam']), axis=1)
attendance_df['points_home'] = points_home
points_away = attendance_df.apply(lambda x: util.calc_point_average_before_game(x['season'], x['matchday'],x['awayteam']), axis=1)
attendance_df['points_away'] = points_away

#Hinzufügen der Position in der Liga:
position_home = attendance_df.apply(lambda x: util.calc_postion_before_game(x['season'], x['matchday'],x['hometeam']), axis=1)
attendance_df['position_home'] = position_home
position_away = attendance_df.apply(lambda x: util.calc_postion_before_game(x['season'], x['matchday'],x['awayteam']), axis=1)
attendance_df['position_away'] = position_away

#Hinzufügen der Position in der vorsaision:
last_season_position_home = attendance_df.apply(lambda x: util.calc_last_seasons_position(x['season'], x['hometeam']), axis=1)
attendance_df['last_season_position_home'] = last_season_position_home
last_season_position_away = attendance_df.apply(lambda x: util.calc_last_seasons_position(x['season'], x['awayteam']), axis=1)
attendance_df['last_season_position_away'] = last_season_position_away

#Hinzufügen der aktuellen Form:
form_home = attendance_df.apply(lambda x: util.calc_point_average_from_last_five_games(x['season'], x['matchday'],x['hometeam'], attendance_df), axis=1)
attendance_df['form_home'] = form_home
form_away = attendance_df.apply(lambda x: util.calc_point_average_from_last_five_games(x['season'], x['matchday'], x['awayteam'], attendance_df), axis=1)
attendance_df['form_away'] = form_away

#Hinzufügen der Wetterdaten
weather_data = attendance_df.apply(lambda x: util.calc_weather_data(x['date'], x['hometeam']), axis=1)
attendance_df[['rain','sun', 'temp_avg']] = pd.DataFrame(weather_data.tolist(), index=weather_data.index)
attendance_df['sun'] = pd.to_numeric(attendance_df['sun'])
attendance_df

,weekday,date,time,stadium,attendance,hometeam,awayteam,matchday,result,season,...,points_away,position_home,position_away,last_season_position_home,last_season_position_away,form_home,form_away,rain,sun,temp_avg
0,Sonntag,2004-03-21,14:30,Stadion Wankdorf,7500,BSC Young Boys,Servette Genève,25,4:2,2003,...,1.750000,2,3,4,6,2.2,1.6,2.5,27,9.7
1,Sonntag,2003-11-23,16:15,Stadion Wankdorf,11000,BSC Young Boys,Servette Genève,17,3:0,2003,...,1.750000,2,3,4,6,3.0,1.4,0,176,5.9
2,Sonntag,2004-02-15,16:00,Stadion Wankdorf,11850,BSC Young Boys,FC Basel,19,0:1,2003,...,2.888889,2,1,4,2,2.6,2.6,0,19,1.8
3,Dienstag,2003-07-22,19:30,Stadion Wankdorf,11850,BSC Young Boys,FC Basel,2,2:3,2003,...,3.000000,1,3,4,2,3.0,3.0,7.8,666,21.4
4,Sonntag,2004-05-09,16:15,Stadion Wankdorf,5000,BSC Young Boys,Neuchâtel Xamax FC,33,3:1,2003,...,0.937500,2,9,4,3,2.4,1.8,2.9,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2855,Mittwoch,2019-05-22,20:00,Letzigrund,100,Grasshopper Club Zürich,FC Sion,35,0:3,2018,...,1.176471,10,8,9,6,0.4,0.6,0,529,13.6
2856,Dienstag,2018-09-25,20:00,Letzigrund,3500,Grasshopper Club Zürich,FC Thun,8,0:2,2018,...,1.571429,8,2,9,7,1.4,1.6,0,678,9.3
2857,Samstag,2019-04-20,19:00,Letzigrund,4300,Grasshopper Club Zürich,FC Thun,30,1:1,2018,...,1.344828,10,3,9,7,0.8,0.4,0,773,15.6
2858,Sonntag,2018-12-09,16:00,Letzigrund,4800,Grasshopper Club Zürich,FC Luzern,17,2:3,2018,...,1.187500,9,7,9,3,1.2,1.4,12.4,22,6


In [22]:
f = open("../data/matches_prep.csv", "w")
attendance_df.to_csv(f, index= False)
f.close()

In [20]:
attendance_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2842 entries, 0 to 2859
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   weekday                    2842 non-null   object        
 1   date                       2842 non-null   datetime64[ns]
 2   time                       2842 non-null   object        
 3   stadium                    2842 non-null   object        
 4   attendance                 2842 non-null   int64         
 5   hometeam                   2842 non-null   object        
 6   awayteam                   2842 non-null   object        
 7   matchday                   2842 non-null   int8          
 8   result                     2842 non-null   object        
 9   season                     2842 non-null   int64         
 10  team_distance              2842 non-null   float64       
 11  points_home                2842 non-null   float64       
 12  points